# John Conway: Game of Life
Notebook-based implementation using ipycanvas

In [1]:
import copy
from ipycanvas import Canvas, hold_canvas
from time import sleep
from ipywidgets import widgets
from IPython.display import display

### Instantiate variables and define the initial states:

In [17]:
cell_size_widget = widgets.IntSlider(min=2, max=20, value=10)
display(cell_size_widget)
cell_border_size_widget = widgets.IntSlider(min=0, max=10, value=0)
display(cell_border_size_widget)

IntSlider(value=10, max=20, min=2)

IntSlider(value=0, max=10)

In [18]:
start = widgets.Dropdown(
    options=[('Simple Oszillator', 'simple_oszillator'), ('Acron', 'acron'), ('Gosper', 'gosper'), ('r_petonmino', 'r_petonmino')],
    value='simple_oszillator',
    description='Starting generation options:',
)
display(start)

Dropdown(description='Starting generation options:', options=(('Simple Oszillator', 'simple_oszillator'), ('Ac…

In [29]:
# instantiate variables
playground_size = (1400, 800)
cell = {}
grid = {}
next_gen = {}
x_array = []
y_array = []
x = y = 0
col = row = 0
cell_size = int(cell_size_widget.value)
cell_border_size = int(cell_size_widget.value)-int(cell_border_size_widget.value)
num_generations = 1000

In [30]:
# build the initial empty grid:
def empty_grid():
    x = y = 0
    for row in range(0,100):
        #print("row (y): ", row)
        for col in range(0,100):
            cell = {"x":x, "y":y, "size":cell_size, "active":0, "row":row, "col":col}
            #print("row (y): ", row, "col (x): ", col, cell)
            grid[(row,col)]=(cell)
            x += cell_size
            col += 1

        x = 0
        col = 0
        y += cell_size

    return grid

In [31]:
def simple_oszillator(grid):
    # define the r-pentomino:
    grid[(5, 1)]['active']=1
    grid[(5, 2)]['active']=1
    grid[(5, 3)]['active']=1

    return grid

In [32]:
def r_petonmino(grid):
    # define the r-pentomino:
    grid[(6, 5)]['active']=1
    grid[(6, 6)]['active']=1
    grid[(7, 4)]['active']=1
    grid[(7, 5)]['active']=1
    grid[(8, 5)]['active']=1

    return grid

In [33]:
def acron(grid):
    grid[(40, 41)]['active']=1
    grid[(41, 43)]['active']=1
    grid[(42, 40)]['active']=1
    grid[(42, 41)]['active']=1
    grid[(42, 44)]['active']=1
    grid[(42, 45)]['active']=1
    grid[(42, 46)]['active']=1

    return grid

In [34]:
def gosper(grid):
    grid[(28, 73)]['active']=1
    grid[(29, 71)]['active']=1
    grid[(29, 73)]['active']=1
    grid[(30, 61)]['active']=1
    grid[(30, 62)]['active']=1
    grid[(30, 69)]['active']=1
    grid[(30, 70)]['active']=1
    grid[(30, 83)]['active']=1
    grid[(30, 84)]['active']=1
    grid[(31, 60)]['active']=1
    grid[(31, 64)]['active']=1
    grid[(31, 69)]['active']=1
    grid[(31, 70)]['active']=1
    grid[(31, 83)]['active']=1
    grid[(31, 84)]['active']=1
    grid[(32, 49)]['active']=1
    grid[(32, 50)]['active']=1
    grid[(32, 59)]['active']=1
    grid[(32, 65)]['active']=1
    grid[(32, 69)]['active']=1
    grid[(32, 70)]['active']=1
    grid[(33, 49)]['active']=1
    grid[(33, 50)]['active']=1
    grid[(33, 59)]['active']=1
    grid[(33, 63)]['active']=1
    grid[(33, 65)]['active']=1
    grid[(33, 66)]['active']=1
    grid[(33, 71)]['active']=1
    grid[(33, 73)]['active']=1
    grid[(34, 59)]['active']=1
    grid[(34, 65)]['active']=1
    grid[(34, 73)]['active']=1
    grid[(35, 60)]['active']=1
    grid[(35, 64)]['active']=1
    grid[(36, 61)]['active']=1
    grid[(36, 62)]['active']=1

    return grid

In [35]:
inital_grid = empty_grid()

if start.value == 'simple_oszillator':
    grid = simple_oszillator(inital_grid)
elif start.value == 'acron':
    grid = acron(inital_grid)
elif start.value == 'gosper':
    grid = gosper(inital_grid)
elif start.value == 'r_petonmino':
    grid = r_petonmino(inital_grid)

# before the first run, copy the initial grid to the next generation dictionary:
next_gen = copy.deepcopy(grid)

### Functions for implementation in a loop:

In [36]:
def next_population(grid):
    # create a copy of the current state of the grid in order not to overwrite the status when moving over the cells:

    for cell in grid.keys():
        # get the x and y postion of the active cell:
        x_pos = cell[0]
        y_pos = cell[1]
    
        nw = grid.get((x_pos-1,y_pos-1),{}).get('active') or 0
        n = grid.get((x_pos,y_pos-1),{}).get('active') or 0
        ne = grid.get((x_pos+1,y_pos-1),{}).get('active') or 0
        w = grid.get((x_pos-1,y_pos),{}).get('active') or 0
        e = grid.get((x_pos+1,y_pos),{}).get('active') or 0
        sw = grid.get((x_pos-1,y_pos+1),{}).get('active') or 0
        s = grid.get((x_pos,y_pos+1),{}).get('active') or 0
        se = grid.get((x_pos+1,y_pos+1),{}).get('active') or 0
    
        res = nw + n + ne + w + e + sw + s + se
        #print("cell: ", x_pos, y_pos, "active: ", grid.get((x_pos,y_pos),{}).get('active'), "nw=", nw, "n=", n, "ne=", ne, "w=", w, "e=", e, "sw=", sw, "s=", s, "se=", se, " result: ", res)
            
        if grid.get((x_pos,y_pos),{}).get('active') == 1 and res < 2:
            next_gen[(x_pos,y_pos)]['active']=0
            #print("result for the cell at position {} = {} the cell dies by underpopulation {}".format(cell, res, next_gen[(x_pos,y_pos)]))
        elif grid.get((x_pos,y_pos),{}).get('active') == 1 and (res == 2 or res == 3):
            next_gen[(x_pos,y_pos)] = grid[cell]
            #print("result for the cell at position {} = {} the cell lives on {}".format(cell, res, next_gen[(x_pos,y_pos)]))
        elif grid.get((x_pos,y_pos),{}).get('active') == 1 and res > 3:
            next_gen[(x_pos,y_pos)]['active']=0
            #print("result for the cell at position {} = {} the cell dies by overpopulation {}".format(cell, res, next_gen[(x_pos,y_pos)]))
        elif grid.get((x_pos,y_pos),{}).get('active') == 0 and res == 3:
            next_gen[(x_pos,y_pos)]['active']=1
            #print("result for the cell at position {} = {} the cell becomes a live cell {}".format(cell, res, next_gen[(x_pos,y_pos)]))
        
        #print(x_pos, y_pos, res, grid[cell])
            
    return next_gen
    #grid = copy.deepcopy(next_gen)

In [37]:
def get_active_cells(grid):
    x_array = []
    y_array = []

    for cell, cell_info in grid.items():
        x_pos, y_pos = cell
        if cell_info.get('active') == 1:
            x_array.append(cell_info.get('x'))
            y_array.append(cell_info.get('y'))

    return x_array, y_array

### Loop to draw the Canvas:

In [38]:
canvas = Canvas(width=playground_size[0], height=playground_size[1])
display(canvas)

# Number of steps in your animation
steps_number = num_generations

for i in range(steps_number):
    with hold_canvas():
        # Clear the old animation step
        canvas.clear()

        # draw the step on the Canvas:
        canvas.font = "16px serif"
        text = "population: "+str(i)
        canvas.fill_text(text, 10, 32)

        # calculate the next population: 
        next_gen = next_population(grid)

        # get the x and y arrays: 
        arrays = get_active_cells(grid)

        # Perfom all your drawings here
        canvas.fill_rects(arrays[0], arrays[1], cell_border_size, height=None)

        # copy the result of the next generation to the 
        grid = copy.deepcopy(next_gen)
        
    # Animation frequency ~50Hz = 1./50. seconds
    #sleep(0.001)
    canvas.sleep(5)

Canvas(height=800, width=1400)

### Cells for testing and Debugging:

In [ ]:
# build the initial population:
#grid[(1,1)] = {'x': 10, 'y': 10, 'size': 10, 'active': 1, 'row': 1, 'col': 1}
#grid[(0, 0)]={'x': 0, 'y': 0, 'size': 10, 'active': 0, 'row': 0, 'col': 0}

# define an oszillator:
#grid[(1, 1)]={'x': 10, 'y': 10, 'size': 9, 'active': 1, 'row': 1, 'col': 1}
#grid[(1, 1)]['active']=1
#grid[(1, 2)]['active']=1
#grid[(1, 3)]['active']=1

In [ ]:
# create a copy of the current state of the grid in order not to overwrite the status when moving over the cells:

for cell in grid.keys():
    # get the x and y postion of the active cell:
    x_pos = cell[0]
    y_pos = cell[1]

    nw = grid.get((x_pos-1,y_pos-1),{}).get('active') or 0
    n = grid.get((x_pos,y_pos-1),{}).get('active') or 0
    ne = grid.get((x_pos+1,y_pos-1),{}).get('active') or 0
    w = grid.get((x_pos-1,y_pos),{}).get('active') or 0
    e = grid.get((x_pos+1,y_pos),{}).get('active') or 0
    sw = grid.get((x_pos-1,y_pos+1),{}).get('active') or 0
    s = grid.get((x_pos,y_pos+1),{}).get('active') or 0
    se = grid.get((x_pos+1,y_pos+1),{}).get('active') or 0

    res = nw + n + ne + w + e + sw + s + se
    #print("cell: ", x_pos, y_pos, "active: ", grid.get((x_pos,y_pos),{}).get('active'), "nw=", nw, "n=", n, "ne=", ne, "w=", w, "e=", e, "sw=", sw, "s=", s, "se=", se, " result: ", res)
        
    if grid.get((x_pos,y_pos),{}).get('active') == 1 and res < 2:
        next_gen[(x_pos,y_pos)]['active']=0
        #print("result for the cell at position {} = {} the cell dies by underpopulation {}".format(cell, res, next_gen[(x_pos,y_pos)]))
    elif grid.get((x_pos,y_pos),{}).get('active') == 1 and (res == 2 or res == 3):
        next_gen[(x_pos,y_pos)] = grid[cell]
        #print("result for the cell at position {} = {} the cell lives on {}".format(cell, res, next_gen[(x_pos,y_pos)]))
    elif grid.get((x_pos,y_pos),{}).get('active') == 1 and res > 3:
        next_gen[(x_pos,y_pos)]['active']=0
        #print("result for the cell at position {} = {} the cell dies by overpopulation {}".format(cell, res, next_gen[(x_pos,y_pos)]))
    elif grid.get((x_pos,y_pos),{}).get('active') == 0 and res == 3:
        next_gen[(x_pos,y_pos)]['active']=1
        #print("result for the cell at position {} = {} the cell becomes a live cell {}".format(cell, res, next_gen[(x_pos,y_pos)]))
    
    #print(x_pos, y_pos, res, grid[cell])

grid = copy.deepcopy(next_gen)


In [ ]:
#grid = copy.deepcopy(next_gen)

In [ ]:
#next_gen

In [ ]:
x_array = []
y_array = []

for cell in grid.keys():
    #print(cell, grid[cell])
    # get the x and y postion of the active cell:
    x_pos = cell[0]
    y_pos = cell[1]

    if grid.get((x_pos,y_pos),{}).get('active'):
        x_array.append(grid.get((x_pos,y_pos),{}).get('x'))
        y_array.append(grid.get((x_pos,y_pos),{}).get('y'))

In [ ]:
canvas = Canvas(width=500, height=800)

#canvas.fill_rect(25, 25, 10)
canvas.fill_rects(x_array, y_array, 9, height=None)

canvas